In [10]:
"""Importing header files"""
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from os.path import exists
from os.path import join
from subprocess import call
from urllib import urlretrieve
from urlparse import urljoin
from Tkinter import *

In [2]:
def download(year, link) :
    """To download the data pdf from website and convert it to .txt format"""
    
    pdfpath = join("pdftext",year+".pdf")
    # Create local path for pdf
    if not exists(pdfpath) :
        urlretrieve(link, pdfpath) #Copy a network object denoted by a URL to a local file, if necessary.
        
    txtpath = pdfpath.replace(".pdf",".txt")
    if not exists(txtpath) :
        call(["pdftotext" ,"-layout", pdfpath, txtpath])

        
download("2014","http://eci.nic.in/eci_main/archiveofge2014/33%20-%20Constituency%20wise%20detailed%20result.pdf")

In [3]:
field = {"2014.txt": ['NAME', 'SEX', 'AGE', 'CATEGORY', 'PARTY', 'SYMBOL', 'GENERAL VOTES', 'POSTAL VOTES', 'VOTES', '% ELECTORS', '% VOTES']}

In [16]:
dictionary = {}

In [17]:
def dataclean (txtfile) :
    re_state = re.compile("^[A-Z][A-Za-z &]+$")
    re_constielectors = re.compile("CONSTITUENCY *: *\d+ *\. *(.*)")
    #re_details = re.compile("\d *[A-Z a-z \.]*")
    
    fieldlist = field[txtfile]
    constituency = None
    electors = None
    state = None
    output = []
    for line in open("pdftext/"+txtfile) :
        match = re_state.match(line)
        if match and len(line)<30:
            state = match.group()
            continue
        
            
        match = re_constielectors.match(line)
        if match :
            constielectors = match.groups()[0].split()
            #constielectors_list = constielectors.split()
            constituency = constielectors[0]
            electors = constielectors[-2]
            #print constituency,electors
            continue
            
            
        details = re.split('  +',line.strip())
        if len(details) == len(fieldlist) :    
            #zipped = zip(fieldlist , details)
            dictionary = dict(zip(fieldlist,details))
            dictionary['NAME'] = ' '.join(dictionary['NAME'].split()[1:])
        else :
            continue
        
        dictionary['STATE'] = state
        dictionary['CONSTITUENCY'] = constituency
        dictionary['YEAR'] = txtfile.split('.')[0]
        output.append(dictionary)
        
    
    df = pd.DataFrame(output).set_index(['STATE','CONSTITUENCY','YEAR'])
    if '%' in df:
        del df['%']
    df.reset_index()
    df.to_csv("2014.csv")
    return output
    


In [11]:
def plot(dictionary) :
    print "5 values will be plotted for Demo Purpose"
    print field.values()
    print ""
    x = raw_input("Give input for X-axis")
    y = raw_input("Give input for Y-axis")
    x = x.upper()
    y = y.upper()
    plt.ylabel(y)
    plt.xlabel(x)
    xaxis = range(5)
    yaxis = []
    xlabels = []
    for d in dictionary[:5] :
        yaxis.append(float(d[y]))
        xlabels.append(d[x])
    plt.bar(xaxis, yaxis, width=0.5, align="center") 
    plt.xticks(xaxis, xlabels)

    plt.show()
    
plot(dataclean("2014.txt"))

5 values will be plotted for Demo Purpose
[['NAME', 'SEX', 'AGE', 'CATEGORY', 'PARTY', 'SYMBOL', 'GENERAL VOTES', 'POSTAL VOTES', 'VOTES', '% ELECTORS', '% VOTES']]

Give input for X-axisCategory
Give input for Y-axisgeneral votes


In [15]:
print field[4]

KeyError: 4